In [1]:
using Pkg
Pkg.DEFAULT_IO[] = stdout
Pkg.status()

     Project demo v0.1.0
      Status `~/Documents/julia/demo/Project.toml`
  [336ed68f] CSV v0.9.6
  [7876af07] Example v0.5.4 `~/.julia/dev/Example`
  [ced58d7a] HighDimMixedModels v0.1.0 `~/.julia/dev/HighDimMixedModels`
  [14b8a8f1] PkgTemplates v0.7.19
  [bd369af6] Tables v1.6.0
  [44d3d7a6] Weave v0.10.10


In [ ]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels
using NLopt
using HighDimMixedModels

In [ ]:
df = DataFrame(MixedModels.dataset(:cbpp))
df[!,:period] = map(x->parse(Float64,x),df[:,:period])
select!(df,[:period,:herd,:incid,:hsz])
first(df,5)

#### parse random effect in formula

In [ ]:
f = @formula(period ~ 0 + incid + hsz + (1|herd))
HMM = highDimMixedModel(f, df, 1)

In [ ]:
sigma, betaM, betaX, opt = fit(HMM, verbose = true, REML = true, alg = :LN_BOBYQA) # :LN_BOBYQA :LN_COBYLA
println("")

## debug opt

In [ ]:
n = size(HMM.M, 1)
A = hcat(HMM.M.M, HMM.X.X)
P = I - A*inv(transpose(A)*A)*transpose(A)
u,s,v = svd(P)
r = size(HMM.M,2) + size(HMM.X,2)  # simplify: assume fixed effect full rank
#C = randn((n-r),n)
C = transpose(u[:,1:(n-r)]) 
K = C*P
Z = HMM.Z.Z
y = HMM.y
# C can be any full rank matrix with size n,r, e.g. randn(n,r)
print()

#### (a) construct Opt by optsum object

In [ ]:
alg = :LN_COBYLA
verbose = true
## add optsum
# init para
sigma = [1.0,1.0]
lbd = [0.0; 0.0]
optsum = OptSummary(Float64.(sigma), lbd, alg; ftol_rel=(1.0e-12), ftol_abs=(1.0e-8), xtol_rel = 1e-5)
optsum.REML = true

## init opt based on optsum
opt = Opt(optsum)

function negLogLik(sigma::Vector{Float64}, g::Vector{Float64})
    n = length(y)
    Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))
    negLog = -1/2*log(det(K*Sigma*transpose(K))) - 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y
    #println("OPT: parameter $(sigma) || objective eval $(negLog)")
    @show sigma
    @show negLog

    return negLog
end

println("The initial object value is $(negLogLik(Float64.(sigma), [1.0,1.0]))")

opt.min_objective = negLogLik
optsum.finitial = negLogLik(optsum.initial, [1.0,1.0])  # the second field not useful right now

In [ ]:
if verbose println("OPTBL: starting point $(sigma)") ; end    # to stdout
(optf,optx,ret) = optimize(opt, sigma)

In [ ]:
# Sigma = optx[1]*Z*transpose(Z) + optx[2]*diagm(ones(n))
# beta = inv(transpose(A)*inv(Sigma)*A)*transpose(A)*inv(Sigma)*y
# betaM = beta[1:size(HMM.M,2)]
# betaX = beta[(size(HMM.M,2) + 1): size(beta)[1]]

# optsum.feval = opt.numevals
# optsum.final = optx
# optsum.fmin = optf
# optsum.returnvalue = ret

# if verbose println("got $(round(optf, digits=5)) at $(round.(optx, digits=5)) after $(opt.numevals) iterations (returned $(ret))") ; end

# HMM.optsum = optsum

#### (b) construct opt by Opt(:LN_COBYLA, 2)

In [ ]:
using NLopt
alg = :LN_COBYLA
verbose = true
## add optsum
# init para
sigma = [1.0,1.0]
lbd = [0.0; 0.0]
optsum = OptSummary(Float64.(sigma), lbd, alg; ftol_rel=(1.0e-12), ftol_abs=(1.0e-8), xtol_rel = 1e-5)
optsum.REML = true

## init opt based on optsum
#opt = Opt(optsum)

## init opt based on Opt function
opt = Opt(:LN_COBYLA, 2)

function negLogLik(sigma::Vector{Float64}, g::Vector{Float64})
    n = length(y)
    Sigma = sigma[1]*Z*transpose(Z) + sigma[2]*diagm(ones(n))
    negLog = -1/2*log(det(K*Sigma*transpose(K))) - 1/2*transpose(y)*transpose(K)*inv(K*Sigma*transpose(K))*K*y
    #println("OPT: parameter $(sigma) || objective eval $(negLog)")
    @show sigma
    @show negLog

    return negLog
end

println("The initial object value is $(negLogLik(Float64.(sigma), [1.0,1.0]))")

opt.min_objective = negLogLik
optsum.finitial = negLogLik(optsum.initial, [1.0,1.0])  # the second field not useful right now

if verbose println("OPTBL: starting point $(sigma)") ; end    # to stdout
(optf,optx,ret) = optimize(opt, sigma)